In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc
gc.enable()
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
#print(os.listdir("../input"))
!ls -GFlash  ../input

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
%%time
# import Dataset to play with it
train_identity= pd.read_csv("C:/Users/shahzadi/Downloads/ieee-fraud-detection/train_identity.csv", index_col='TransactionID')
train_transaction= pd.read_csv("C:/Users/shahzadi/Downloads/ieee-fraud-detection/train_transaction.csv", index_col='TransactionID')
test_identity= pd.read_csv("C:/Users/shahzadi/Downloads/ieee-fraud-detection/test_identity.csv", index_col='TransactionID')
test_transaction = pd.read_csv('C:/Users/shahzadi/Downloads/ieee-fraud-detection/test_transaction.csv', index_col='TransactionID')
print ("Done!")

Done!
Wall time: 5min 22s
Parser   : 128 ms


In [4]:
print('Shape of Data:')
print(train_transaction.shape)
print(test_transaction.shape)
print(train_identity.shape)
print(test_identity.shape)

Shape of Data:
(590540, 393)
(506691, 392)
(144233, 40)
(141907, 40)


In [24]:
# Creat our train & test dataset
#%%time
train = train_transaction.merge(train_identity, how='left', left_index=True, right_index=True)
test = test_transaction.merge(test_identity, how='left', left_index=True, right_index=True)

In [25]:
del train_identity,train_transaction,test_identity, test_transaction

In [ ]:
train.info()

In [ ]:
test.info()

In [7]:
#Based on this great kernel https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65
def reduce_mem_usage(df):
    start_mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in df.columns:
        if df[col].dtype != object:  # Exclude strings            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",df[col].dtype)            
            # make variables for Int, max and min
            IsInt = False
            mx = df[col].max()
            mn = df[col].min()
            print("min for this col: ",mn)
            print("max for this col: ",mx)
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(df[col]).all(): 
                NAlist.append(col)
                df[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = df[col].fillna(0).astype(np.int64)
            result = (df[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        df[col] = df[col].astype(np.uint8)
                    elif mx < 65535:
                        df[col] = df[col].astype(np.uint16)
                    elif mx < 4294967295:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)    
            # Make float datatypes 32 bit
            else:
                df[col] = df[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",df[col].dtype)
            print("******************************")
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = df.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return df, NAlist

In [8]:
train, NAlist = reduce_mem_usage(train)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 1975.3707885742188  MB
******************************
Column:  isFraud
dtype before:  int64
min for this col:  0
max for this col:  1
dtype after:  uint8
******************************
******************************
Column:  TransactionDT
dtype before:  int64
min for this col:  86400
max for this col:  15811131
dtype after:  uint32
******************************
******************************
Column:  TransactionAmt
dtype before:  float64
min for this col:  0.251
max for this col:  31937.391
dtype after:  float32
******************************
******************************
Column:  card1
dtype before:  int64
min for this col:  1000
max for this col:  18396
dtype after:  uint16
******************************
******************************
Column:  card2
dtype before:  float64
min for this col:  100.0
max for this col:  600.0
dtype after:  uint16
******************************
******************************
Column:  card3
dtype before:  float64


dtype after:  uint8
******************************
******************************
Column:  V10
dtype before:  float64
min for this col:  0.0
max for this col:  4.0
dtype after:  uint8
******************************
******************************
Column:  V11
dtype before:  float64
min for this col:  0.0
max for this col:  5.0
dtype after:  uint8
******************************
******************************
Column:  V12
dtype before:  float64
min for this col:  0.0
max for this col:  3.0
dtype after:  uint8
******************************
******************************
Column:  V13
dtype before:  float64
min for this col:  0.0
max for this col:  6.0
dtype after:  uint8
******************************
******************************
Column:  V14
dtype before:  float64
min for this col:  0.0
max for this col:  1.0
dtype after:  uint8
******************************
******************************
Column:  V15
dtype before:  float64
min for this col:  0.0
max for this col:  7.0
dtype after:  ui

dtype after:  uint8
******************************
******************************
Column:  V60
dtype before:  float64
min for this col:  0.0
max for this col:  16.0
dtype after:  uint8
******************************
******************************
Column:  V61
dtype before:  float64
min for this col:  0.0
max for this col:  6.0
dtype after:  uint8
******************************
******************************
Column:  V62
dtype before:  float64
min for this col:  0.0
max for this col:  10.0
dtype after:  uint8
******************************
******************************
Column:  V63
dtype before:  float64
min for this col:  0.0
max for this col:  7.0
dtype after:  uint8
******************************
******************************
Column:  V64
dtype before:  float64
min for this col:  0.0
max for this col:  7.0
dtype after:  uint8
******************************
******************************
Column:  V65
dtype before:  float64
min for this col:  0.0
max for this col:  1.0
dtype after:  

dtype after:  uint8
******************************
******************************
Column:  V110
dtype before:  float64
min for this col:  0.0
max for this col:  7.0
dtype after:  uint8
******************************
******************************
Column:  V111
dtype before:  float64
min for this col:  0.0
max for this col:  9.0
dtype after:  uint8
******************************
******************************
Column:  V112
dtype before:  float64
min for this col:  0.0
max for this col:  9.0
dtype after:  uint8
******************************
******************************
Column:  V113
dtype before:  float64
min for this col:  0.0
max for this col:  9.0
dtype after:  uint8
******************************
******************************
Column:  V114
dtype before:  float64
min for this col:  0.0
max for this col:  6.0
dtype after:  uint8
******************************
******************************
Column:  V115
dtype before:  float64
min for this col:  0.0
max for this col:  6.0
dtype afte

dtype after:  float32
******************************
******************************
Column:  V160
dtype before:  float64
min for this col:  0.0
max for this col:  641511.4375
dtype after:  float32
******************************
******************************
Column:  V161
dtype before:  float64
min for this col:  0.0
max for this col:  3300.0
dtype after:  float32
******************************
******************************
Column:  V162
dtype before:  float64
min for this col:  0.0
max for this col:  3300.0
dtype after:  float32
******************************
******************************
Column:  V163
dtype before:  float64
min for this col:  0.0
max for this col:  3300.0
dtype after:  float32
******************************
******************************
Column:  V164
dtype before:  float64
min for this col:  0.0
max for this col:  93736.0
dtype after:  float32
******************************
******************************
Column:  V165
dtype before:  float64
min for this col:  0.0


dtype after:  float32
******************************
******************************
Column:  V209
dtype before:  float64
min for this col:  0.0
max for this col:  8050.0
dtype after:  float32
******************************
******************************
Column:  V210
dtype before:  float64
min for this col:  0.0
max for this col:  3300.0
dtype after:  float32
******************************
******************************
Column:  V211
dtype before:  float64
min for this col:  0.0
max for this col:  92888.0
dtype after:  float32
******************************
******************************
Column:  V212
dtype before:  float64
min for this col:  0.0
max for this col:  129006.0
dtype after:  float32
******************************
******************************
Column:  V213
dtype before:  float64
min for this col:  0.0
max for this col:  97628.0
dtype after:  float32
******************************
******************************
Column:  V214
dtype before:  float64
min for this col:  0.0
ma

dtype after:  uint8
******************************
******************************
Column:  V258
dtype before:  float64
min for this col:  0.0
max for this col:  66.0
dtype after:  uint8
******************************
******************************
Column:  V259
dtype before:  float64
min for this col:  0.0
max for this col:  285.0
dtype after:  uint16
******************************
******************************
Column:  V260
dtype before:  float64
min for this col:  0.0
max for this col:  8.0
dtype after:  uint8
******************************
******************************
Column:  V261
dtype before:  float64
min for this col:  0.0
max for this col:  49.0
dtype after:  uint8
******************************
******************************
Column:  V262
dtype before:  float64
min for this col:  0.0
max for this col:  20.0
dtype after:  uint8
******************************
******************************
Column:  V263
dtype before:  float64
min for this col:  0.0
max for this col:  153600.0

dtype after:  float32
******************************
******************************
Column:  V307
dtype before:  float64
min for this col:  0.0
max for this col:  145765.0
dtype after:  float32
******************************
******************************
Column:  V308
dtype before:  float64
min for this col:  0.0
max for this col:  108800.0
dtype after:  float32
******************************
******************************
Column:  V309
dtype before:  float64
min for this col:  0.0
max for this col:  55125.0
dtype after:  float32
******************************
******************************
Column:  V310
dtype before:  float64
min for this col:  0.0
max for this col:  55125.0
dtype after:  float32
******************************
******************************
Column:  V311
dtype before:  float64
min for this col:  0.0
max for this col:  55125.0
dtype after:  float32
******************************
******************************
Column:  V312
dtype before:  float64
min for this col:  0.0

min for this col:  100.0
max for this col:  661.0
dtype after:  uint16
******************************
******************************
Column:  id_21
dtype before:  float64
min for this col:  100.0
max for this col:  854.0
dtype after:  uint16
******************************
******************************
Column:  id_22
dtype before:  float64
min for this col:  10.0
max for this col:  44.0
dtype after:  uint8
******************************
******************************
Column:  id_24
dtype before:  float64
min for this col:  11.0
max for this col:  26.0
dtype after:  uint8
******************************
******************************
Column:  id_25
dtype before:  float64
min for this col:  100.0
max for this col:  548.0
dtype after:  uint16
******************************
******************************
Column:  id_26
dtype before:  float64
min for this col:  100.0
max for this col:  216.0
dtype after:  uint8
******************************
******************************
Column:  id_32
dtyp

In [9]:
test, NAlist = reduce_mem_usage(test)
print("_________________")
print("")
print("Warning: the following columns have missing values filled with 'df['column_name'].min() -1': ")
print("_________________")
print("")
print(NAlist)

Memory usage of properties dataframe is : 1693.867820739746  MB
******************************
Column:  TransactionDT
dtype before:  int64
min for this col:  18403224
max for this col:  34214345
dtype after:  uint32
******************************
******************************
Column:  TransactionAmt
dtype before:  float64
min for this col:  0.018000000000000002
max for this col:  10270.0
dtype after:  float32
******************************
******************************
Column:  card1
dtype before:  int64
min for this col:  1001
max for this col:  18397
dtype after:  uint16
******************************
******************************
Column:  card2
dtype before:  float64
min for this col:  100.0
max for this col:  600.0
dtype after:  uint16
******************************
******************************
Column:  card3
dtype before:  float64
min for this col:  100.0
max for this col:  232.0
dtype after:  uint8
******************************
******************************
Column:  card5


dtype after:  uint8
******************************
******************************
Column:  V11
dtype before:  float64
min for this col:  0.0
max for this col:  7.0
dtype after:  uint8
******************************
******************************
Column:  V12
dtype before:  float64
min for this col:  0.0
max for this col:  4.0
dtype after:  uint8
******************************
******************************
Column:  V13
dtype before:  float64
min for this col:  0.0
max for this col:  6.0
dtype after:  uint8
******************************
******************************
Column:  V14
dtype before:  float64
min for this col:  0.0
max for this col:  1.0
dtype after:  uint8
******************************
******************************
Column:  V15
dtype before:  float64
min for this col:  0.0
max for this col:  13.0
dtype after:  uint8
******************************
******************************
Column:  V16
dtype before:  float64
min for this col:  0.0
max for this col:  25.0
dtype after:  

dtype after:  uint8
******************************
******************************
Column:  V61
dtype before:  float64
min for this col:  0.0
max for this col:  6.0
dtype after:  uint8
******************************
******************************
Column:  V62
dtype before:  float64
min for this col:  0.0
max for this col:  10.0
dtype after:  uint8
******************************
******************************
Column:  V63
dtype before:  float64
min for this col:  0.0
max for this col:  8.0
dtype after:  uint8
******************************
******************************
Column:  V64
dtype before:  float64
min for this col:  0.0
max for this col:  10.0
dtype after:  uint8
******************************
******************************
Column:  V65
dtype before:  float64
min for this col:  0.0
max for this col:  1.0
dtype after:  uint8
******************************
******************************
Column:  V66
dtype before:  float64
min for this col:  0.0
max for this col:  8.0
dtype after:  

dtype after:  uint8
******************************
******************************
Column:  V111
dtype before:  float64
min for this col:  0.0
max for this col:  8.0
dtype after:  uint8
******************************
******************************
Column:  V112
dtype before:  float64
min for this col:  0.0
max for this col:  8.0
dtype after:  uint8
******************************
******************************
Column:  V113
dtype before:  float64
min for this col:  0.0
max for this col:  8.0
dtype after:  uint8
******************************
******************************
Column:  V114
dtype before:  float64
min for this col:  0.0
max for this col:  9.0
dtype after:  uint8
******************************
******************************
Column:  V115
dtype before:  float64
min for this col:  0.0
max for this col:  9.0
dtype after:  uint8
******************************
******************************
Column:  V116
dtype before:  float64
min for this col:  0.0
max for this col:  9.0
dtype afte

dtype after:  float32
******************************
******************************
Column:  V160
dtype before:  float64
min for this col:  0.0
max for this col:  3867868.5
dtype after:  float32
******************************
******************************
Column:  V161
dtype before:  float64
min for this col:  0.0
max for this col:  2000.0
dtype after:  float32
******************************
******************************
Column:  V162
dtype before:  float64
min for this col:  0.0
max for this col:  4000.0
dtype after:  float32
******************************
******************************
Column:  V163
dtype before:  float64
min for this col:  0.0
max for this col:  4000.0
dtype after:  float32
******************************
******************************
Column:  V164
dtype before:  float64
min for this col:  0.0
max for this col:  928882.0
dtype after:  float32
******************************
******************************
Column:  V165
dtype before:  float64
min for this col:  0.0
m

dtype after:  float32
******************************
******************************
Column:  V209
dtype before:  float64
min for this col:  0.0
max for this col:  10000.0
dtype after:  float32
******************************
******************************
Column:  V210
dtype before:  float64
min for this col:  0.0
max for this col:  4000.0
dtype after:  float32
******************************
******************************
Column:  V211
dtype before:  float64
min for this col:  0.0
max for this col:  928882.0
dtype after:  float32
******************************
******************************
Column:  V212
dtype before:  float64
min for this col:  0.0
max for this col:  958320.0
dtype after:  float32
******************************
******************************
Column:  V213
dtype before:  float64
min for this col:  0.0
max for this col:  928882.0
dtype after:  float32
******************************
******************************
Column:  V214
dtype before:  float64
min for this col:  0.0

dtype after:  uint8
******************************
******************************
Column:  V258
dtype before:  float64
min for this col:  0.0
max for this col:  269.0
dtype after:  uint16
******************************
******************************
Column:  V259
dtype before:  float64
min for this col:  0.0
max for this col:  131.0
dtype after:  uint8
******************************
******************************
Column:  V260
dtype before:  float64
min for this col:  0.0
max for this col:  14.0
dtype after:  uint8
******************************
******************************
Column:  V261
dtype before:  float64
min for this col:  0.0
max for this col:  89.0
dtype after:  uint8
******************************
******************************
Column:  V262
dtype before:  float64
min for this col:  0.0
max for this col:  34.0
dtype after:  uint8
******************************
******************************
Column:  V263
dtype before:  float64
min for this col:  0.0
max for this col:  106549

dtype after:  float32
******************************
******************************
Column:  V307
dtype before:  float64
min for this col:  0.0
max for this col:  958320.0
dtype after:  float32
******************************
******************************
Column:  V308
dtype before:  float64
min for this col:  0.0
max for this col:  718740.0
dtype after:  float32
******************************
******************************
Column:  V309
dtype before:  float64
min for this col:  0.0
max for this col:  64800.0
dtype after:  float32
******************************
******************************
Column:  V310
dtype before:  float64
min for this col:  0.0
max for this col:  167200.0
dtype after:  float32
******************************
******************************
Column:  V311
dtype before:  float64
min for this col:  0.0
max for this col:  64800.0
dtype after:  float32
******************************
******************************
Column:  V312
dtype before:  float64
min for this col:  0.

dtype after:  uint16
******************************
******************************
Column:  id_22
dtype before:  float64
min for this col:  11.0
max for this col:  44.0
dtype after:  uint8
******************************
******************************
Column:  id_24
dtype before:  float64
min for this col:  10.0
max for this col:  26.0
dtype after:  uint8
******************************
******************************
Column:  id_25
dtype before:  float64
min for this col:  100.0
max for this col:  549.0
dtype after:  uint16
******************************
******************************
Column:  id_26
dtype before:  float64
min for this col:  100.0
max for this col:  216.0
dtype after:  uint8
******************************
******************************
Column:  id_32
dtype before:  float64
min for this col:  8.0
max for this col:  48.0
dtype after:  uint8
******************************
___MEMORY USAGE AFTER COMPLETION:___
Memory usage is:  480.99015617370605  MB
This is  28.39596751791695

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 2987000 to 3577539
Columns: 433 entries, isFraud to DeviceInfo
dtypes: float32(80), int16(7), int8(9), object(31), uint16(63), uint32(2), uint8(241)
memory usage: 568.5+ MB


In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506691 entries, 3663549 to 4170239
Columns: 432 entries, TransactionDT to DeviceInfo
dtypes: float32(80), int16(1), int8(9), object(31), uint16(50), uint32(2), uint8(259)
memory usage: 481.0+ MB


In [10]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [20]:
os.getcwd()

'C:\\Users\\shahzadi'